In [12]:
from dask.distributed import LocalCluster, Client
import time
import dask.array as da

cluster = LocalCluster()
client = Client(cluster)



C:\Users\abdeljelil.maadi\.conda\envs\dask-examples\lib\site-packages\distributed\node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50371 instead
  warnings.warn(


In [13]:
from dask.distributed import Client, progress

client = Client(n_workers=2, threads_per_worker=2, memory_limit='2GB')
client

C:\Users\abdeljelil.maadi\.conda\envs\dask-examples\lib\site-packages\distributed\node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50416 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:50417 Dashboard: http://127.0.0.1:50416/status,Cluster Workers: 2 Cores: 4 Memory: 4.00 GB


In [43]:
# from sklearn.datasets import load_iris

# iris = load_iris()
# X, y = iris.data, iris.target

import dask.dataframe as dd
import pandas as pd
df = dd.read_csv('iris.csv')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [44]:
features = [col for col in df.columns if col not in ["variety", "sepal_length"]]
label = "variety"

In [45]:
# create column transformer
numeric_features = df[features].select_dtypes(include=["int64", "float64"]).columns
categorical_features = df[features].select_dtypes(include=["category"]).columns
df = df.categorize(columns=categorical_features)

In [46]:
from dask_ml.model_selection import train_test_split
## train test split

X_train, X_test, y_train, y_test = train_test_split(
 df[features], df[label], test_size=0.25, random_state=2,
 shuffle=True)

In [47]:
# # persist data to each worker
from dask.distributed import Client, wait
import dask_ml.datasets
import dask_ml.cluster
import dask

from dask_ml.preprocessing import StandardScaler, DummyEncoder
X_train, X_test, y_train, y_test = dask.persist(X_train, X_test, y_train, y_test)
_ = wait(X_train)

In [48]:
from sklearn.pipeline import Pipeline
from dask_ml.impute import SimpleImputer
from dask_ml.preprocessing import StandardScaler, DummyEncoder
numeric_transformer = Pipeline(
 steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

In [49]:
from dask_ml.compose import ColumnTransformer
categorical_transformer = DummyEncoder()
preprocessor = ColumnTransformer(
 transformers=[
 ("num", numeric_transformer, numeric_features),
 ("cat", categorical_transformer, categorical_features),
 ]
)

In [51]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
pipelines = []
rf = Pipeline(
 steps=[
 ("preprocessor", preprocessor),
 ("RF", RandomForestClassifier(random_state=2, class_weight="balanced")),
 ]
)

xgb = Pipeline(
 steps=[
 ("preprocessor", preprocessor),
 ("XGB", GradientBoostingClassifier(random_state=2)),
 ]
)

pipelines = [rf, xgb]

import dask
pipelines_ = [dask.delayed(pl).fit(X_train, y_train) for pl in pipelines]

fit_pipelines = dask.compute(*pipelines_)